In [111]:
from numpy import genfromtxt
import csv
import sys
from keras.layers import Dense
from keras.models import Sequential
from keras.regularizers import l2, activity_l2

sys.path.append('../../')

from NeuroLinguisticDecoding.src.WordEmbeddingLayer import *





brain_activations_1 = genfromtxt('../data/data.csv', delimiter=',')
"""
brain_activations_2 = genfromtxt('../data/data_2.csv', delimiter=',')
brain_activations_3 = genfromtxt('../data/data_3.csv', delimiter=',')
brain_activations_4 = genfromtxt('../data/data_4.csv', delimiter=',')
brain_activations_5 = genfromtxt('../data/data_5.csv', delimiter=',')
brain_activations_6 = genfromtxt('../data/data_6.csv', delimiter=',')
brain_activations_7 = genfromtxt('../data/data_7.csv', delimiter=',')
brain_activations_8 = genfromtxt('../data/data_8.csv', delimiter=',')
brain_activations_9 = genfromtxt('../data/data_9.csv', delimiter=',')
"""

"\nbrain_activations_2 = genfromtxt('../data/data_2.csv', delimiter=',')\nbrain_activations_3 = genfromtxt('../data/data_3.csv', delimiter=',')\nbrain_activations_4 = genfromtxt('../data/data_4.csv', delimiter=',')\nbrain_activations_5 = genfromtxt('../data/data_5.csv', delimiter=',')\nbrain_activations_6 = genfromtxt('../data/data_6.csv', delimiter=',')\nbrain_activations_7 = genfromtxt('../data/data_7.csv', delimiter=',')\nbrain_activations_8 = genfromtxt('../data/data_8.csv', delimiter=',')\nbrain_activations_9 = genfromtxt('../data/data_9.csv', delimiter=',')\n"

In [112]:
print(brain_activations_1[:,:19750].shape)
print(brain_activations_2[:,:19750].shape)
print(brain_activations_3[:,:19750].shape)
print(brain_activations_4[:,:19750].shape)
print(brain_activations_5[:,:19750].shape)
print(brain_activations_6[:,:19750].shape)
print(brain_activations_7[:,:19750].shape)
print(brain_activations_8[:,:19750].shape)
print(brain_activations_9[:,:19750].shape)

brain_activations = brain_activations_1
"""np.concatenate((brain_activations_1[:,:19750],
                        brain_activations_2[:,:19750],
                        brain_activations_3[:,:19750],
                        brain_activations_4[:,:19750],
                        brain_activations_5[:,:19750],
                        brain_activations_6[:,:19750],
                        brain_activations_7[:,:19750],
                        brain_activations_8[:,:19750],
                        brain_activations_9[:,:19750]))"""


print(brain_activations.shape)

(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 19750)
(360, 21764)


In [117]:
words_1 = []
with open('../data/words', 'r') as f:
    reader = csv.reader(f)
    words_1 = list(reader)
    
words = []
words.extend(words_1)


wem = WordEmbeddingLayer()
wem.load_filtered_embedding("../data/neuro_words")

embedded_words = wem.embed(words)

In [118]:
print(len(embedded_words[0][0]))
a = [ e[0] for e in embedded_words]
print(brain_activations[0].shape[0])

300
21764


In [130]:
model = Sequential()
model.add(Dense(input_dim = 300 ,output_dim=1000,_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
model.add(Dense(input_dim = 1000,output_dim=brain_activations[0].shape[0],_regularizer=l2(0.01), activity_regularizer=activity_l2(0.01)))
model.compile("rmsprop","mse",metrics=['accuracy'])

In [120]:
embedded_words_2 = np.asarray([ e[0] for e in embedded_words])
print(embedded_words_2.shape)
model.fit(embedded_words_2,brain_activations,nb_epoch=100)


def calculate_accuracy(model,testX,testY):
    Y = model.predict(testX)

(360, 300)
Epoch 1/100
360/360 [==============================] - 47s - loss: 4.5595 - acc: 0.3083    
Epoch 2/100
360/360 [==============================] - 46s - loss: 0.3486 - acc: 0.6389    
Epoch 3/100
360/360 [==============================] - 47s - loss: 0.3302 - acc: 0.6528    
Epoch 4/100
360/360 [==============================] - 47s - loss: 0.3349 - acc: 0.6500    
Epoch 5/100
360/360 [==============================] - 709s - loss: 0.3345 - acc: 0.6472    
Epoch 6/100
360/360 [==============================] - 51s - loss: 0.3322 - acc: 0.6306    
Epoch 7/100
360/360 [==============================] - 48s - loss: 0.3626 - acc: 0.5667    
Epoch 8/100
360/360 [==============================] - 51s - loss: 0.3398 - acc: 0.6306    
Epoch 9/100
360/360 [==============================] - 49s - loss: 0.4000 - acc: 0.5639    
Epoch 10/100
360/360 [==============================] - 51s - loss: 0.3251 - acc: 0.6250    
Epoch 11/100
360/360 [==============================] - 51s - loss:

KeyboardInterrupt: 

In [137]:
word_set = list(set([word[0] for word in words_1]))

print(len(word_set))
pairs = []
for i in np.arange(60):
    for j in np.arange(i):
        pairs.append((i,j))

all_activations = []
all_features = []
the_pairs = []

60


In [138]:
for (i,j) in pairs:
    activations = []
    the_pairs.append((i,j))
    word_vectors = []
    for k in np.arange(brain_activations.shape[0]):
        if words[k][0] != word_set[i] and words[k][0] != word_set[j]:
            activations.append(brain_activations[k])
            word_vectors.append(embedded_words_2[k])
    all_activations.append(activations)
    all_features.append(word_vectors)

In [139]:
print(len(all_activations))
print(len(all_features))
print(len(the_pairs))

print(len(all_features[0]))
print(embedded_words_2[10].shape)

1770
1770
1770
348
(300,)


In [140]:
model.fit(np.asarray(all_features[0]),np.asarray(all_activations[0]),nb_epoch=2)

Epoch 1/2
348/348 [==============================] - 4s - loss: 0.3630 - acc: 0.2328     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.3071 - acc: 0.6437     


In [146]:
print(embedded_words_2[the_pairs[0][0]].shape)
predicted_1 = model.predict(np.asarray([embedded_words_2[the_pairs[0][0]]]))
predicted_2 = model.predict(np.asarray([embedded_words_2[the_pairs[0][1]]]))

(300,)


In [183]:
from scipy import *
from scipy.spatial.distance import *


def match_prediction(predicted,pair,item):
    cosin_1 = []
    cosin_2 = []
    for i in np.arange(len(words)):
        if words[i][0] == word_set[pair[item]]:
            cosin = cosine(predicted,brain_activations[i])
            cosin_1.append(cosin)
        if words[i][0] == word_set[pair[(item + 1) %2]]:
            cosin = cosine(predicted,brain_activations[i])
            cosin_2.append(cosin)
    
    return sum(np.asarray([c1-c2 for c1 in cosin_1 for c2 in cosin_2]) < 0) > (0.5*len(cosin_1)*len(cosin_2))


print(match_prediction(predicted_1[0],the_pairs[0],0))
print(match_prediction(predicted_2[0],the_pairs[0],1))

True
False


In [ ]:
result = []
for k in np.arange(len(the_pairs)):
    model.reset_states()
    model.fit(np.asarray(all_features[k]),np.asarray(all_activations[k]),nb_epoch=2)
    predicted_1 = model.predict(np.asarray([embedded_words_2[the_pairs[k][0]]]))
    predicted_2 = model.predict(np.asarray([embedded_words_2[the_pairs[k][1]]]))
    res1 = match_prediction(predicted_1[0],the_pairs[k],0)
    res2 = match_prediction(predicted_2[0],the_pairs[k],1)
    result.append(res1)
    result.append(res2)

Epoch 1/2
348/348 [==============================] - 3s - loss: 0.2695 - acc: 0.6523     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.2695 - acc: 0.6494     
Epoch 1/2
348/348 [==============================] - 3s - loss: 0.2682 - acc: 0.6437     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.2686 - acc: 0.6466     
Epoch 1/2
348/348 [==============================] - 3s - loss: 0.2687 - acc: 0.6523     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.2684 - acc: 0.6552     
Epoch 1/2
348/348 [==============================] - 3s - loss: 0.2690 - acc: 0.6523     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.2687 - acc: 0.6523     
Epoch 1/2
348/348 [==============================] - 3s - loss: 0.2692 - acc: 0.6523     
Epoch 2/2
348/348 [==============================] - 3s - loss: 0.2690 - acc: 0.6552     
Epoch 1/2
348/348 [==============================] - 4s - loss: 0.2680 - acc: 0.6437     
Epoch 2/2


True
